# 1. Import libraries

In [2]:
import pandas as pd
from pathlib import Path
from collections import Counter

# 2. EDA

In [3]:
# Check out what is in train.csv
train_df = pd.read_csv("../data/global-wheat-detection/train.csv")
print(f"train.csv shape {train_df.shape}")
train_df.head()

train.csv shape (147793, 5)


,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [4]:
# Look at unique values in each column
for col in train_df.columns.to_list():
    display(train_df[col].value_counts())

image_id
35b935b6c    116
f1794c924    108
f79336a8e    104
134d4a01c    101
4cf91995d     97
            ... 
86296fc32      1
ce4124be0      1
4e6c05213      1
a5cb30c38      1
41457a646      1
Name: count, Length: 3373, dtype: int64

width
1024    147793
Name: count, dtype: int64

height
1024    147793
Name: count, dtype: int64

bbox
[972, 0, 52, 84]              2
[0.0, 957.0, 79.0, 66.0]      2
[372.0, 856.0, 66.0, 64.0]    2
[302.0, 815.0, 64.0, 75.0]    2
[0.0, 385.0, 56.0, 46.0]      2
                             ..
[0, 750, 39, 78]              1
[121, 931, 119, 77]           1
[279, 965, 91, 59]            1
[588, 953, 81, 71]            1
[875.0, 740.0, 94.0, 61.0]    1
Name: count, Length: 147761, dtype: int64

source
ethz_1       51489
arvalis_1    45716
rres_1       20236
arvalis_3    16665
usask_1       5807
arvalis_2     4179
inrae_1       3701
Name: count, dtype: int64

```Observations```

1. 3373 images, all 1024 x 1024 px
2. Probably can disregard source
3. I will need to draw the bounding boxes into each image id

In [5]:
# Check number of images in train and test folder
train_dir = "../data/global-wheat-detection/train"
test_dir = "../data/global-wheat-detection/test"

train_img = [image for image in Path(train_dir).iterdir()]
train_img_ext_count = Counter([image.suffix for image in train_img])
print(f"Images in train folder: {len(train_img)}")
print(f"Image suffixes: {train_img_ext_count}")

print()

test_img = [image for image in Path(test_dir).iterdir()]
test_img_ext_count = Counter([image.suffix for image in test_img])
print(f"Images in test folder: {len(test_img)}")
print(f"Image suffixes: {test_img_ext_count}")

print(dir(train_img))

Images in train folder: 3422
Image suffixes: Counter({'.jpg': 3422})

Images in test folder: 10
Image suffixes: Counter({'.jpg': 10})
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


```Observations```

1. There are 3422 images in train folder, but only 3733 unique IDs in train.csv.
    - Explore further to see which unique IDs do not have images in train folder
2. All .jpg extension in train folder. Great

In [ ]:
# Investigate missing IDs in train.csv
## Check which images in train folder is not present in unique ID in train.csv
train_img_name = [image.stem for image in train_img]
train_csv_id = train_df["image_id"].unique()

print(f"No. of unique train image names: {len(set(train_img_name))}")
print(f"No. of unique image_ids: {len(set(train_csv_id))}")


3422
3373
['75d62c43e', 'a7436f5c7', '3049b3745', 'ef274ecdc', '6895c563e', 'fb9e42d30', '2c28cb6dd', '499bf6abd', 'f165314f1', '564832c17', '769309456', '5a61c3e15', '86e843e5e', '2643476a0', '49c3e4f6e', '2f2716b45', 'd145202eb', '7ebf61d5a', '2204976da', '82f88bab4', 'f290ed4e6', 'b5e7db94b', '9f2963463', '4b74f7320', '0cbcaf240', '8f796ab69', '69286a6cc', '382bf4ed7', 'ec6d53041', 'e08c90ebe', 'b5487f037', 'ba98d009b', 'f9cafa47f', '90184122e', '0adcb2a1a', 'e525a27f8', 'cdc68ad10', '1f3829014', '483f5b46c', 'b900ac004', '28c948177', '7c2780e55', '2e1ed4321', '9bedf6645', '2d8a88333', 'f0665430d', '4a2662fae', 'c34dd05a3', '015939012', '5f9cfe156', 'e840ecd4e', 'c742fa9ae', '0352a6fff', 'ac5bb0dfa', '92f3b252f', '56ace8298', 'aa171bd87', '20c7cc731', '9694225d3', '78cf5b3fd', 'fc8cf268b', '90d0bcc83', '509a7a9b5', 'ba586e514', '9181010cc', 'a23558330', '1b352d5fe', 'd13e3ffce', '3e0213b6f', '6a2eac670', '4af27998f', '809f14a6a', '059125072', '598e8cea3', '035bdc07a', 'b3c96d5ad', '